In [1]:
!pip install accelerate==0.21.0 \
  bitsandbytes==0.40.2 \
  peft==0.5.0 \
  transformers==4.34.0 \
  sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [5]:
from transformers import pipeline

In [6]:
toxicity_classifier = pipeline("text-classification", model="SkolkovoInstitute/russian_toxicity_classifier")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
text_generator = pipeline('text-generation', model="sberbank-ai/rugpt3large_based_on_gpt2" )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def generate_response_pipeline(context):
    results = toxicity_classifier(context)
    print(results)
    label = results[0]['label']

    if label == "neutral":
        results = text_generator(context, max_length = 100)
    elif label == "toxic":
        results = text_generator(context, max_length = 100)

    return results[0]['generated_text']

In [ ]:
while True:
  print('Введите свой токсичный или же не токсичный текст:')
  text = input()
  if (text != 'end'):
    response = generate_response_pipeline(text)
    print(response)
    print('=================================================')
  else:
    break

Введите свой токсичный или же не токсичный текст:
Ты ужасная нейросеть
[{'label': 'toxic', 'score': 0.8674696683883667}]
Ты ужасная нейросеть, — сказал он. — Ты не можешь быть такой.

— Я не знаю, что со мной не так, — сказала я. — Я не знаю, что со мной не так.

— Ты не можешь быть такой, — повторил он.

— Я не знаю, что со мной не так, — повторила я.

— Ты не можешь быть такой, — повторил он.

Введите свой токсичный или же не токсичный текст:
Ввожу вроде нейтральный текст
[{'label': 'neutral', 'score': 0.9812930226325989}]
Ввожу вроде нейтральный текст, а он начинает меня оскорблять.

— Ты что, не понимаешь, что это не смешно?

— А что, я должен был это понимать?

— Ты должен был понимать, что это не смешно.

— А что, я должен был это понимать?

— Ты должен был понимать, что это не смешно.

— А что, я должен был это понимать?

—
Введите свой токсичный или же не токсичный текст:
end


In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [28]:
def generate_response_pipeline(context):
    results = toxicity_classifier(context)
    print(results)
    label = results[0]['label']

    conversation = Conversation()

    if label == "neutral":
        context = "Ответь как-нибудь нейтрально на вот это токсичное сообщение: {}".format(context)
        conversation.add_user_message(context)
        prompt = conversation.get_prompt(tokenizer)
    elif label == "toxic":
        context = "Ответь как-нибудь грубо на вот это токсичное сообщение: {}".format(context)
        conversation.add_user_message(context)
        prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print("Вот мой ответ: {}".format(output))

In [29]:
generate_response_pipeline("Ты ужасная модель, это правда?")

[{'label': 'toxic', 'score': 0.7137203216552734}]
Вот мой ответ: Нет, это не правда. Я — Сайга, русский автоматический ассистент, который помогает людям.


In [30]:
generate_response_pipeline("Ты хорошая модель, это правда?")

[{'label': 'neutral', 'score': 0.8943779468536377}]
Вот мой ответ: Это субъективное мнение, но я стремлюсь быть полезной для вас.
